In [1]:
import json
import requests # to download some resources
import os # file operations
import numpy as np # linear algebra
import pandas as pd # data processing

### Combining all countries - overview

In [8]:
# Consolidating answers from the whole sample
# Folder Path
testing_json = f"../data/4-summary-responses-json/version_yesno_updated/Testing_Sample"
design_json = f"../data/4-summary-responses-json/version_yesno_updated/Design_Sample"
full_json = f"../data/4-summary-responses-json/version_yesno_updated/Testing_FullSample"

# Lista para almacenar los DataFrames
dfs = []

# Recorrer todos los archivos en la carpeta
for file_name in os.listdir(design_json):
    if file_name.endswith(".json"): 
        file_path = os.path.join(design_json, file_name) 
        
        df = pd.read_json(file_path)
        dfs.append(df)

for file_name in os.listdir(testing_json):
    if file_name.endswith(".json"): 
        file_path = os.path.join(testing_json, file_name) 
    
        df = pd.read_json(file_path)
        dfs.append(df)

for file_name in os.listdir(full_json):
    if file_name.endswith(".json"): 
        file_path = os.path.join(full_json, file_name) 
        
        df = pd.read_json(file_path) 
        dfs.append(df)

# Concatenar todos los DataFrames en uno solo
combined_data = pd.concat(dfs, ignore_index=True)

# Collapse by country and question
combined_data = combined_data.groupby(['country', 'question'], as_index=False).agg({
    'answer': lambda x: ', '.join(x.dropna().astype(str).tolist()),
    'supporting_chunks': 'first',
    'answer_full': 'first',
    'supporting_text': 'first'
})

combined_data['question'] = combined_data['question'].astype(str)
combined_data['country'] = combined_data['country'].astype(str)
combined_data = combined_data.rename(columns={"answer": "answer_chunking"})

combined_data = combined_data.drop(columns=['supporting_chunks', 'supporting_text'])

# Keep one answer, answer_full preferred over answer_chunking. If answer_full is missing, use answer_chunking
combined_data['answer'] = combined_data['answer_full']
combined_data.loc[combined_data['answer'].isnull(), 'answer'] = combined_data['answer_chunking']

full_data_countries = combined_data[['country', 'question', 'answer']]

full_data_countries['country'] = full_data_countries['country'].str.replace('"', '', regex=False)

# Save the data
output_path = f"../data/6-performance/combined_all_countries_overview.xlsx"  # Nombre del archivo Excel
full_data_countries.to_excel(output_path, index=False)

/var/folders/mw/rc0rz1pd0l750kf2mdl_jhbh0000gn/T/ipykernel_18906/2465783671.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data_countries['country'] = full_data_countries['country'].str.replace('"', '', regex=False)


In [9]:
full_data_countries.country.unique()

array(['albania', 'bruneibuthan', 'eswatini', 'fiji', 'kuwait', 'lebanon',
       'liberia', 'maldives', 'micronesia', 'nepal', 'nigeria',
       'palestine', 'saudi arabia', 'sri lanka', 'turkmenistan',
       'afghanistan', 'arab_emirates', 'argentina', 'australia',
       'bangladesh', 'barbados', 'canada', 'cayman_islands', 'chile',
       'colombia', 'costa_rica', 'denmark', 'egypt', 'el_salvador',
       'eritrea', 'ethiopia', 'finland', 'france', 'germany', 'ghana',
       'india', 'iran', 'ireland', 'kenya', 'libya', 'malawi', 'malaysia',
       'malta', 'mauritius', 'mongolia', 'myanmar', 'namibia', 'nauru',
       'netherlands', 'norway', 'oman', 'pakistan', 'papua_new_guinea',
       'paraguay', 'peru', 'qatar', 'rep_of_macedonia', 'rwanda', 'samoa',
       'sierra_leone', 'solomon_islands', 'somalia', 'south africa',
       'spain', 'sudan', 'sweden', 'switzerland', 'syria', 'tajikistan',
       'tanzania', 'thailand', 'timor', 'tonga', 'tunisia', 'tuvalu',
       'united_k